# Creation of database, table and assignment of privileges
- The database is named `crimes_db`, the table is named `boston_crimes` which is wrapped in the schema named `crimes`.
- The rows in the table are filled with the data from `boston.csv` file
- Finally, two groups `readonly` and `readwrite` are created and one user is assigned for each group, with respective privileges defined.

In [91]:
""" create a database and schema"""
import psycopg2
conn = psycopg2.connect(dbname ='dq',user='dq')
conn.autocommit = True
cur=conn.cursor()
#cur.execute('CREATE DATABASE crime_db') # Avoid re-creation
conn.autocommit = False
conn.close()


In [92]:
"""create database, crime_db"""
conn= psycopg2.connect(dbname ='crime_db',user='dq')
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes") #avoid re-creation

In [93]:
"""Read the first two lines from the boston.csv file"""
import csv
with open("boston.csv", 'r') as f:
    reader = list(csv.reader(f))
col_headers = reader[0]
first_row = reader[1]
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [94]:
"""define get_col_set(): given a file and index,
    returns all the entries in a column specified by the index """
def get_col_set(csv_filename, col_index):
    col_set = set()
    with open(csv_filename) as f:
        next(f)
        rows = csv.reader(f)
        for row in rows:
            col_set.add(row[col_index])
    return col_set
for i in range(7):
    print(len(get_col_set("boston.csv",i)))
# column 5, (i.e. index 4) has lowest distinct members,
# hence a good candidate for enumerated datatypes             

298329
219
239
1177
7
18177
18177


In [95]:
"""Determine the maximum length of each value in the description column"""
print(col_headers)
print(first_row)
col_values = get_col_set("boston.csv",2)
# which value has maximum length?
max_val = 0
max_str = " "
for entry in col_values:
    if len(entry)>max_val:
        max_val = len(entry)
        max_str = entry
print(max_str)
print(max_val)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']
RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUTSIDE BOSTON)
58


In [96]:
enum_val = get_col_set("boston.csv",4)
print(enum_val)
q = """ CREATE TYPE enum_weekday AS ENUM (
'Sunday','Tuesday', 'Saturday', 'Thursday','Friday','Wednesday','Monday'); """
cur.execute(q)




{'Sunday', 'Tuesday', 'Saturday', 'Thursday', 'Friday', 'Wednesday', 'Monday'}


# Data Type Choices
- For the column `day_of_the_week`, enumerated datatype is used because the values are few.
- The `lat` and `long` columns use floats because the values are about 8 decimal places
- The `description` column is assigned variable character length

In [97]:
q1 = """CREATE TABLE crimes.boston_crimes(
        incident_number integer PRIMARY KEY,
        offence_code integer,
        description varchar(100),
        date date,
        day_of_the_week enum_weekday,
        lat float,
        long float)"""
cur.execute(q1)

In [98]:
"""Load data into the table: boston_crimes"""
with open("boston.csv", mode= "r") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;",f)

In [99]:
"""REVOKE all default privileges and GRANT privileges to new groups"""
cur.execute("""REVOKE ALL ON SCHEMA public FROM public; """)
cur.execute("""REVOKE ALL ON DATABASE crime_db FROM public; """)
cur.execute("""CREATE GROUP readonly NOLOGIN;""")
cur.execute("""CREATE GROUP readwrite NOLOGIN;""")
cur.execute(""" GRANT connect ON DATABASE crime_db TO readonly;""")
cur.execute(""" GRANT connect ON DATABASE crime_db TO readwrite;""")
cur.execute("""GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly""")
cur.execute("""GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA 
crimes TO readwrite""")

In [100]:
""" Create Users data_analyst and data_scientist"""
cur.execute("""CREATE USER data_analyst WITH CREATEDB PASSWORD 'secret1';""")
cur.execute("""GRANT readonly TO data_analyst;""")
cur.execute("""CREATE USER data_scientist WITH CREATEDB PASSWORD 'secret2';""")
cur.execute("""GRANT readwrite TO data_scientist;""")

In [101]:
"""Check if the privileges are assigned appropriately"""
q3 = """SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';"""
cur.execute(q3)
cur.fetchall()

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

In [104]:
cur.execute("""SELECT rolname FROM pg_roles""")
cur.

AttributeError: 'psycopg2._psycopg.cursor' object has no attribute 'table'